In [18]:
import os, nltk
os.getcwd()

'c:\\Users\\T-Gamer\\Documents\\SideDrive\\UFMA\\2022.1\\Topicos Especiais (NLP)\\Exercicios\\Trabalho Final\\Implementação\\source'

In [30]:
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\T-Gamer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [71]:
import pandas as pd
dataset_folder = "../resources/datasets/TwitterAirlines"
dataset = pd.read_csv(f"{dataset_folder}/Tweets.csv", index_col=0)

In [53]:
columns_to_keep = ["airline_sentiment", "airline_sentiment_confidence", "text"]
dataset_clean = dataset.iloc[ : , [column in columns_to_keep for column in dataset.columns]]

In [72]:
dataset_clean.to_csv(f"{dataset_folder}/TweetsDroppedCols.csv")

In [63]:
def remove_ats_hashtags_and_links(text:str) -> str :
    gross_tokenlist = text.split()
    text = ""
    for gross_token in gross_tokenlist :
        if not (gross_token.startswith("@") or
                gross_token.startswith("#") or
                gross_token.startswith("http")) :
            text += gross_token + " "
    return text[:-1]
def remove_stopwords(tokenlist:list[str]) -> list[str] :
    stoplist = stopwords.words('english')
    newtokenlist = []
    for token in tokenlist :
        if not token in stoplist :
            newtokenlist.append(token)
    return newtokenlist
def remove_nonalpha(tokenlist:list[str]) -> list[str] :
    newtokenlist = []
    for tkn in tokenlist :
        newtkn = ""
        for char in tkn :
            if char.isalpha() :
                newtkn += char
        if newtkn != "" : newtokenlist.append(newtkn)
    return newtokenlist
def preprocess(text:str) -> str :
    text = remove_ats_hashtags_and_links(text)
    text = text.lower()
    tokenlist = nltk.word_tokenize(text)
    tokenlist = remove_stopwords(tokenlist)
    lemmatizer = nltk.WordNetLemmatizer()
    lemmalist = [lemmatizer.lemmatize(token) for token in tokenlist]
    lemmalist = remove_nonalpha(lemmalist)
    text = ""
    for tkn in lemmalist :
        text += tkn + " "
    text = text[ : -1]
    return text

In [55]:
dataset_clean = dataset_clean.copy() # Defrag

In [83]:
processed_text = []
classes = []
index = []
for i, (_, row) in enumerate(dataset_clean.iterrows()) :
    if i % 100 == 0 : 
        print(i)
    processed = preprocess(row["text"])
    if processed == "" : processed = " "
    processed_text.append(processed)
    classes.append(["negative", "neutral", "positive"].index(row["airline_sentiment"]))
    index.append(row.name)
processed_text_column = pd.Series(processed_text, index, name="text")
classes_column = pd.Series(classes, index, name="class")

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600


In [84]:
dataset_processed = pd.concat([processed_text_column, classes_column], axis=1)

In [85]:
dataset_processed

,text,class
570306133677760513,said,1
570301130888122368,plus ve added commercial experience tacky,2
570301083672813571,nt today must mean need take another trip,1
570301031407624196,s really aggressive blast obnoxious entertainm...,0
570300817074462722,s really big bad thing,0
...,...,...
569587686496825344,thank got different flight chicago,2
569587371693355008,leaving minute late flight warning communicati...,0
569587242672398336,please bring american airline,1
569587188687634433,money change flight nt answer phone suggestion...,0


In [86]:
dataset_processed = dataset_processed.dropna()

In [87]:
dataset_processed.to_csv(f"{dataset_folder}/TweetsProcessed.csv")